In [7]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.agents import load_tools, initialize_agent, AgentType, create_react_agent, AgentExecutor
import os
from dotenv import load_dotenv
from IPython.display import Markdown

### Laod environment variables and utility methods

In [8]:
load_dotenv()

# clear the old OPENAI_API_BASE environment variable to avoid error
del os.environ['OPENAI_API_BASE'] 
def show_result(result):
    display(Markdown(f"##### Question : \n{result['input']}"))
    display(Markdown(f"##### Answer : \n {result['output']}"))

## Intial Setup
Create AzureChatOpenAI LLM model
Load 'Bing' search tool

In [9]:
llm = AzureChatOpenAI(azure_deployment='gpt-4'
                      ,model_name="gpt-4"
                      ,temperature=0, verbose=True)

tools = load_tools(["bing-search"], llm=llm)
question = "Who won 2024 super bowl? What was score and who was MVP?"

## Part I - Bing Search 
Search answer using stable version of Agents

In [10]:

agent= initialize_agent(tools,llm,agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
response = agent.invoke (question)

show_result(response)



c:\Users\louisli\AppData\Local\miniconda3\envs\openai_v1\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I don't have this information readily available, so I'll use the bing_search tool to find out who won the 2024 Super Bowl, what the score was, and who was named MVP.
Action:
```
{
  "action": "bing_search",
  "action_input": "2024 Super Bowl winner score and MVP"
}
```

Search using latest library

In [18]:
prompt_template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(prompt_template)
tools = load_tools(["bing-search"], llm=llm)
agent1 = create_react_agent(llm, tools,prompt)
agent_executor = AgentExecutor(agent=agent1, tools=tools,verbose=True)
response = agent_executor.invoke({"input": question})

show_result(response)




> Entering new AgentExecutor chain...
I need to use the Bing search tool to find out the most recent information on the 2024 Super Bowl winner, the final score, and the MVP of the game.

Action: bing_search
Action Input: 2024 Super Bowl winner, final score, MVPThe Kansas City Chiefs defeated the San Francisco 49ers 25-22 in a thrilling overtime <b>Super</b> <b>Bowl</b> victory, cementing head coach Andy Reid and quarterback Patrick Mahomes’ team as the new NFL... Patrick Mahomes is the <b>Super</b> <b>Bowl</b> <b>2024</b> <b>MVP</b>!. Mahomes, 28, won the trophy following the Kansas City Chiefs&#39; 25-22 victory over the San Francisco 49ers, making it the third time in his career that ... Ruby Franke sentenced Audrii Cunningham February 12, <b>2024</b> 8:34 AM PST How the Chiefs shape their roster will be a top story to follow in the offseason 3:44 AM PST Mahomes, the Chiefs, Taylor Swift and a thrilling <b>game</b> -- it all came together at the <b>Super</b> <b>Bowl</b> February 11

##### Question : 
Who won 2024 super bowl? What was score and who was MVP?

##### Answer : 
 The Kansas City Chiefs won the 2024 Super Bowl with a final score of 25-22 against the San Francisco 49ers. The MVP of the game was Patrick Mahomes.

## Part 2 - Search from specific site
Now lets search query on specific site

In [19]:
site_question = "(site:https://www.fda.gov/food) Whar is sampling? Explain process, purpose and how it is used in food safety?"
response = agent_executor.invoke({"input": site_question})

show_result(response)



> Entering new AgentExecutor chain...
To answer this question, I need to find information on the FDA's official website regarding the definition of sampling, the process involved, its purpose, and how it is used in food safety. Since the question specifies the site (https://www.fda.gov/food), I will use the Bing Search tool to find the relevant information on that website.

Action: bing_search
Action Input: site:https://www.fda.gov/food what is sampling in food safetySurveillance <b>sampling</b> is just one type of <b>sampling</b> that is important for <b>food</b> <b>safety</b>. There are different kinds of <b>sampling</b>, and the methods that the FDA uses fall into three broad categories ... Establishments may be identified for environmental <b>sampling</b> using this process, which employs criteria related to <b>food</b>-hazard pairs (e.g., frequency of outbreaks associated with a <b>food</b>, likelihood ... Identify each <b>sample</b> unit (defined later) with a properly marked s

##### Question : 
(site:https://www.fda.gov/food) Whar is sampling? Explain process, purpose and how it is used in food safety?

##### Answer : 
 Sampling in food safety is the process of collecting samples of food products, in-process items, and raw ingredients to test for safety and quality. The purpose of sampling is to prevent food safety issues by identifying potential hazards and ensuring that food products do not reach consumers with harmful levels of contaminants or pathogens. The FDA employs various types of sampling, such as environmental sampling, which uses criteria related to food-hazard pairs to identify establishments for testing. Sampling is an integral part of the FDA's preventive approach to food safety, as mandated by the FDA Food Safety Modernization Act. HACCP principles guide the systematic identification, evaluation, and control of food safety hazards, which are closely related to the sampling process.